# Imports

In [901]:
import requests
import bs4
import json
import datetime as dt
import sys
import pandas as pd
import geopy
import warnings
warnings.filterwarnings('ignore')

# Lisbon Properties for Sale - SUPERCASA Webscrapping

In [907]:
url = 'https://supercasa.pt/comprar-casas/lisboa/pagina-1'
result = requests.get(url)
soup = bs4.BeautifulSoup(result.text, 'lxml')
num_of_properties = int(soup.find_all('h1', id='searchTitle')[0].get_text().split()[0].replace('.',''))
num_prop_per_page = 25
total_pages = int(num_of_properties / num_prop_per_page)
time_estimation_for_12505 = 20
time_estimation = round(num_of_properties * 20 / 12505)

print(f'Time estimation for this task: {time_estimation} minutes.')
estimation = (dt.datetime.now() + dt.timedelta(minutes=time_estimation))
start = f'Tasks started at {dt.datetime.now().hour}h:{dt.datetime.now().minute}min. Estimated finish time {estimation.hour}h:{estimation.minute}min.'
print(start)

title, price, num_rooms, total_area, latitude, longitude, region, extras, id = ([] for i in range(9))

for n in range(1, 75):
    url = f'https://supercasa.pt/comprar-casas/lisboa/pagina-{n}'
    result = requests.get(url)
    soup = bs4.BeautifulSoup(result.text, 'lxml')

    # Find all properties on the current page
    properties = soup.find_all('div', class_='property big-picture') 

    for prop in properties:
        # Title
        a = prop.find('h2', class_='property-list-title').find('a')
        title.append(a.get_text().strip() if a else ' ')

        # Price
        span = prop.find('div', class_='property-price').find('span')
        price.append(span.get_text(strip=True) if span else ' ')

        # Features
        feature = prop.find('div', class_='property-features')
        spans = feature.find_all('span') if feature else []
        rooms = spans[0].get_text() if len(spans) > 0 else "Unknown"
        area = spans[1].get_text() if len(spans) > 1 else "Unknown"
        num_rooms.append(rooms)
        total_area.append(area)

        # Links for latitude and longitude
        link = prop.find('a', class_='property-link')
        latitude.append(link.get('data-latitude') if link else 'Unknown')
        longitude.append(link.get('data-longitude') if link else 'Unknown')

        # Extras
        highlight = prop.find('div', class_='property-highlights')
        if highlight:
            extra_spans = highlight.find_all('span')
            extras.append(', '.join([span.get_text(strip=True) for span in extra_spans]))
        else:
            extras.append(' ')

        # Address region from JSON-LD script if necessary
        script = prop.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            if data.get('@type') == 'Offer':
                available_at_or_from = data.get('availableAtOrFrom', {})
                address_info = available_at_or_from.get('address', {})
                address_region = address_info.get('addressRegion', 'Not provided')
                region.append(address_region)
        else:
            region.append('Not provided')

        sys.stdout.write(f"\rProgress: {int((n / total_pages) * 100)}%")
        sys.stdout.flush()

for i in range(len(title)):
    id.append(i)

sys.stdout.write(f"\rProgress: 100%")
sys.stdout.flush()
print('\nCompleted!')

headers = ['id', 'title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'region', 'extras']
final_data = [id, title, price, num_rooms, total_area, latitude, longitude, region, extras]
Lisbon_Properties = pd.DataFrame(dict(zip(headers, final_data)))

print(f"\nYou now have data on {len(lisbon['id'])} properties located in Lisbon!")

Time estimation for this task: 20 minutes.
Tasks started at 18h:33min. Estimated finish time 18h:53min.
Progress: 100%
Completed!

You now have data on 9408 properties located in Lisbon!


In [940]:
# Lisbon_Properties.to_csv('Lisbon_Properties.csv')

In [919]:
Lisbon_Properties = pd.read_csv('Lisbon_Properties.csv', index_col=0)
Lisbon_Properties

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
1,1,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
2,2,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T1 em Praça de Luís de Camões, Mis...",690.000 €,1 quarto,Área bruta 93 m²,"38,71078","-9,14385",Misericórdia,Com garagem
...,...,...,...,...,...,...,...,...,...
1376,1376,Apartamento T2 na Rua do Vale de Santo António...,399.900 €,2 quartos,Área bruta 55 m²,"38,7177917","-9,1224923",São Vicente,
1377,1377,"Apartamento T4 em Arroios, Lisboa",385.000 €,4 quartos,Área bruta 90 m²,"38,72422","-9,1345992",Arroios,
1378,1378,"Apartamento T4 na Avenida Barbosa Du Bocage, A...",1.700.000 €,4 quartos,Área bruta 177 m²,"38,74006","-9,14737",Avenidas Novas,Com elevador
1379,1379,"Apartamento T1 em Estrada de Benfica, Benfica,...",275.000 €,1 quarto,Área bruta 70 m²,"38,75286","-9,20801",Benfica,


In [941]:
Lisbon_Properties = Lisbon_Properties[~Lisbon_Properties.drop('id', axis=1).duplicated()]

In [928]:
Lisbon_Properties['num_rooms'].value_counts()

num_rooms
2 quartos     486
1 quarto      337
3 quartos     272
4 quartos     136
5 quartos      29
9 quartos      13
7 quartos       6
6 quartos       6
10 quartos      2
8 quartos       1
11 quartos      1
Name: count, dtype: int64

In [929]:
Lisbon_Properties['total_area'].unique()

array(['Área bruta 97 m²', 'Área bruta 90 m²', 'Área bruta 120 m²',
       'Área útil 233 m²', 'Área bruta 93 m²', 'Área bruta 416 m²',
       'Área bruta 55 m²', 'Área bruta 234 m²', 'Área bruta 290 m²',
       'Área bruta 262 m²', 'Área bruta 336 m²', 'Área bruta 264 m²',
       'Área bruta 60 m²', 'Área bruta 874 m²', 'Área bruta 68 m²',
       'Área útil 88 m²', 'Área bruta 48 m²', 'Área bruta 110 m²',
       'Área bruta 102 m²', 'Área útil 142 m²', 'Área bruta 76 m²',
       'Área bruta 119 m²', 'Área bruta 160 m²', 'Área bruta 54 m²',
       'Área bruta 302 m²', 'Área bruta 44 m²', 'Área bruta 150 m²',
       'Área bruta 168 m²', 'Área bruta 70 m²', 'Área bruta 49 m²',
       'Área bruta 230 m²', 'Área bruta 191 m²', 'Área bruta 426 m²',
       'Área bruta 138 m²', 'Área útil 139 m²', 'Área bruta 198 m²',
       'Área bruta 85 m²', 'Área bruta 130 m²', 'Área bruta 180 m²',
       'Área bruta 67 m²', 'Área bruta 134 m²', 'Área bruta 286 m²',
       'Área bruta 77 m²', 'Área bruta 

In [914]:
remove_room_errors = Lisbon_Properties['num_rooms'].apply(lambda x: x.split()[0]).isin([str(i) for i in range(1, 15)])

In [886]:
Lisbon_Properties = Lisbon_Properties[remove_room_errors]

In [920]:
remove_area_errors = Lisbon_Properties['total_area'].apply(lambda x: 'Área' in x)

In [917]:
Lisbon_Properties = Lisbon_Properties[remove_area_errors]

In [889]:
Lisbon_Properties

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
1,1,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
2,2,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem"
...,...,...,...,...,...,...,...,...,...
9870,9870,"Apartamento T4 na Rua Jacinto Nunes, Penha de ...",330.000 €,4 quartos,Área bruta 99 m²,"38,73106","-9,12899",Penha de França,
9871,9871,"Apartamento T2 Duplex em Calçada da Ajuda, Aju...",325.000 €,2 quartos,Área bruta 48 m²,"38,70531","-9,19939",Ajuda,
9872,9872,"Apartamento T1+2 na Rua Cecílio de Sousa, Mise...",320.000 €,1 quarto,Área bruta 68 m²,"38,7160237","-9,1500739",Misericórdia,Rés do chão
9873,9873,"Apartamento T2+1 em Arroios, Lisboa",250.000 €,2 quartos,Área útil 88 m²,"38,7321733","-9,1350438",Arroios,Com elevador


# Lisbon Metro Info - Wikipedia Webscrapping

In [677]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_esta%C3%A7%C3%B5es_do_Metropolitano_de_Lisboa'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
data = str(soup.find('table', {'class': 'wikitable'}))

table = pd.read_html(data)[0]
columns = ['Nome','Outros nomes','Linha','Lat.','Long.']
Lisbon_Metro = table[columns]

# Data

In [838]:
Lisbon_Properties.head()

,id,title,price,num_rooms,total_area,latitude,longitude,region,extras
0,0,"Apartamento T3 na Rua António Nobre, São Domin...",399.900 €,3 quartos,Área bruta 120 m²,"38,74657","-9,17989",São Domingos de Benfica,
1,1,"Apartamento T1 em Benfica, Lisboa",269.900 €,1 quarto,Área bruta 97 m²,"38,75171","-9,2009",Benfica,
2,2,"Apartamento T2 em Alvalade, Lisboa",430.000 €,2 quartos,Área bruta 90 m²,"38,7457392","-9,1425898",Alvalade,
3,3,"Apartamento T4+1 na Rua de São Paulo, Misericó...",1.650.000 €,4 quartos,Área útil 233 m²,"38,70794","-9,14468",Misericórdia,"Com elevador, Com garagem"
4,4,"Apartamento T2+1 em Calçada do Combro, Miseric...",1.800.000 €,2 quartos,Área bruta 234 m²,"38,71101","-9,14865",Misericórdia,"Com elevador, Com garagem"


In [839]:
def extra_rooms(x):
    if '+' in x:
        return int(x.split('+')[1])
    else:
        return 0

def area(x):
    if 'Área' in x:
        return int(x.split()[2].replace('.',''))
    else:
        return 'Unknown'

def num_extras(x):
    if x.isspace():
        return 0
    elif ',' in x:
        return int(len(x.split(',')))
    else:
        return 1

In [890]:
Lisbon_Properties['Type'] = Lisbon_Properties['title'].apply(lambda x: x.split()[0])
Lisbon_Properties['Typology'] = Lisbon_Properties['title'].apply(lambda x: x.split()[1])
Lisbon_Properties['Extra_Rooms'] = Lisbon_Properties['Typology'].apply(extra_rooms)
Lisbon_Properties['Extra_Rooms_Flag'] = Lisbon_Properties['Extra_Rooms'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['N_Rooms'] = Lisbon_Properties['num_rooms'].apply(lambda x: int(x.split()[0]))
Lisbon_Properties['Total_N_Rooms'] = Lisbon_Properties['N_Rooms'] + Lisbon_Properties['Extra_Rooms']
Lisbon_Properties['Price'] = Lisbon_Properties['price'].apply(lambda x: int(x.replace('.','').split()[0]))
Lisbon_Properties['Area_m2'] = Lisbon_Properties['total_area'].apply(area)
Lisbon_Properties['N_Extras'] = Lisbon_Properties['extras'].apply(num_extras)
Lisbon_Properties['Extras_Flag'] = Lisbon_Properties['N_Extras'].apply(lambda x: 1 if x > 0 else 0)
Lisbon_Properties['Latitude'] = Lisbon_Properties['latitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties['Longitude'] = Lisbon_Properties['longitude'].apply(lambda x: float(x.replace(',','.')))
Lisbon_Properties.rename(columns={'region':'Region'}, inplace=True)

In [891]:
extras_df = Lisbon_Properties['extras'].str.split(',', expand=True)
extras_df

,0,1,2,3,4
0,,None,None,None,None
1,,None,None,None,None
2,,None,None,None,None
3,Com elevador,Com garagem,None,None,None
4,Com elevador,Com garagem,None,None,None
...,...,...,...,...,...
9870,,None,None,None,None
9871,,None,None,None,None
9872,Rés do chão,None,None,None,None
9873,Com elevador,None,None,None,None


In [892]:
extras_df = Lisbon_Properties['extras'].str.split(',', expand=True)

extras = []
unique_extras = []

for x in [0,1,2,3,4]:
    extras.append(list(extras_df[x].unique()))

for ext in extras:
    unique_extras += ext

unique_extras = set(unique_extras)
unique_extras = {extra.strip() for extra in unique_extras if extra is not None and extra.strip()}

In [908]:
unique_extras

{'Com elevador',
 'Com garagem',
 'De luxo',
 'Piscina',
 'Rés do chão',
 'Vista para mar',
 'Último andar'}

In [893]:
for feature in unique_extras:
    Lisbon_Properties[feature] = Lisbon_Properties['extras'].apply(lambda x: int(feature in x))

In [894]:
columns_to_drop = ['title', 'price', 'num_rooms', 'total_area', 'latitude', 'longitude', 'Typology','extras']
Lisbon_Properties = Lisbon_Properties.drop(columns_to_drop, axis=1)

In [895]:
Lisbon_Properties.head()

,id,Region,Type,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,Longitude,Vista para mar,Com elevador,De luxo,Piscina,Com garagem,Rés do chão,Último andar
0,0,São Domingos de Benfica,Apartamento,0,0,3,3,399900,120,0,0,38.746570,-9.17989,0,0,0,0,0,0,0
1,1,Benfica,Apartamento,0,0,1,1,269900,97,0,0,38.751710,-9.20090,0,0,0,0,0,0,0
2,2,Alvalade,Apartamento,0,0,2,2,430000,90,0,0,38.745739,-9.14259,0,0,0,0,0,0,0
3,3,Misericórdia,Apartamento,1,1,4,5,1650000,233,2,1,38.707940,-9.14468,0,1,0,0,1,0,0
4,4,Misericórdia,Apartamento,1,1,2,3,1800000,234,2,1,38.711010,-9.14865,0,1,0,0,1,0,0


In [896]:
region_dummies = pd.get_dummies(Lisbon_Properties['Region'],dtype=int)
type_dummies = pd.get_dummies(Lisbon_Properties['Type'],dtype=int)

In [897]:
Lisbon_Properties = pd.concat([Lisbon_Properties, region_dummies, type_dummies], axis=1)

In [898]:
Lisbon_Properties.drop(['Region','Type'], axis=1, inplace=True)

In [899]:
Lisbon_Properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9783 entries, 0 to 9874
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       9783 non-null   int64  
 1   Extra_Rooms              9783 non-null   int64  
 2   Extra_Rooms_Flag         9783 non-null   int64  
 3   N_Rooms                  9783 non-null   int64  
 4   Total_N_Rooms            9783 non-null   int64  
 5   Price                    9783 non-null   int64  
 6   Area_m2                  9783 non-null   int64  
 7   N_Extras                 9783 non-null   int64  
 8   Extras_Flag              9783 non-null   int64  
 9   Latitude                 9783 non-null   float64
 10  Longitude                9783 non-null   float64
 11  Vista para mar           9783 non-null   int64  
 12  Com elevador             9783 non-null   int64  
 13  De luxo                  9783 non-null   int64  
 14  Piscina                  9783

In [900]:
Lisbon_Properties.head()

,id,Extra_Rooms,Extra_Rooms_Flag,N_Rooms,Total_N_Rooms,Price,Area_m2,N_Extras,Extras_Flag,Latitude,...,São Vicente,Apartamento,Casa,Duplex,Flat,Loft,Moradia,Palacete,Penthouse,Quinta
0,0,0,0,3,3,399900,120,0,0,38.746570,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,1,1,269900,97,0,0,38.751710,...,0,1,0,0,0,0,0,0,0,0
2,2,0,0,2,2,430000,90,0,0,38.745739,...,0,1,0,0,0,0,0,0,0,0
3,3,1,1,4,5,1650000,233,2,1,38.707940,...,0,1,0,0,0,0,0,0,0,0
4,4,1,1,2,3,1800000,234,2,1,38.711010,...,0,1,0,0,0,0,0,0,0,0


<html>
<p><strong>Great Circle Distance Formula:</strong></p>
<p>The formula to calculate the great circle or 'as the crow flies' distance between two points on the Earth's surface, given their latitude and longitude is:</p>
<p style="font-family: 'Lucida Console', Monaco, monospace;">
  \( d = 2R \times \sin^{-1}\left(\sqrt{\sin^2\left(\frac{\theta_2 - \theta_1}{2}\right) + \cos \theta_1 \times \cos \theta_2 \times \sin^2\left(\frac{\phi_2 - \phi_1}{2}\right)}\right) \)
</p>
<p>where:</p>
<ul>
  <li><strong>\( (\theta_1, \phi_1) \)</strong> and <strong>\( (\theta_2, \phi_2) \)</strong> – Coordinates of each point (latitude and longitude, respectively);</li>
  <li><strong>R</strong> – Radius of the Earth; and</li>
  <li><strong>d</strong> – Great circle distance between the points.</li>
</l>
</html>


In [902]:
Lisbon_Metro.head()

,Nome,Outros nomes,Linha,Lat.,Long.
0,Aeroporto,—,Vermelha,38.76861,−9.12861
1,Alameda,Alameda I (técn.),Verde,38.73713,−9.13388
2,Alameda,Alameda II (técn.),Vermelha,38.73697,−9.13261
3,Alfornelos,—,Azul,38.76038,−9.20435
4,Alto dos Moinhos,Centro Administrativo (prev.),Azul,38.74994,−9.18003
